<h2>Objective</h2>

<h3> the main objectuve of this notebook is to make use tranfer learning to make a working protoype model of image classification on the small sample images of plants.</h3>

**evaluation metric**

<u>if the we can make a small dataset from the whole dataset and train any model on basis of it .</u>

then we can use those functions to train various models and find out the best model for our dataset.

In [1]:
# ignoring all the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# importing the necessary libraries

import numpy as np
import pandas as pd
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2023-04-16 13:47:55.524894: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 13:47:57.846607: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 13:47:57.850332: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 13:48:02.537251: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# importing the metadata.csv with all the path to images and their labels

tdata=pd.read_csv(r'/home/drowsy/Downloads/data/PlantCLEF2022_trusted_training_metadata.csv',delimiter=';')
wdata=pd.read_csv(r'/home/drowsy/Downloads/data/PlantCLEF2022_web_training_metadata.csv',delimiter=';')

In [ ]:
# directory of the images to train the model.

data_dir='/home/drowsy/Downloads/data/train/images'

# converting the path to pathlib for easy anvigation

import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:
# concating the classid and species of the images
tdata['labels']=tdata['classid'].astype('str')+'$'+tdata['species'].astype('str')

In [ ]:
labels_df2=tdata['labels'].unique()
type(labels_df2)

In [ ]:
# making a dict with classid as key and name of plant species as value

def get_labels(labels):
    label_dict={}
    for i in range(len(labels)):
        label_dict[str(labels[i]).split('$')[0]]=str(labels[i]).split('$')[1]
    return label_dict

plant_all_label_dict=get_labels(labels_df2)
plant_all_label_dict

In [ ]:
# making a dict with name of folder as key 
# and putting the path of all images inside the folder as a list in the dict as values.

def get_images1(labels):
    images_dict={}
#     labels are the list of all the path of subfolders in directory
#     labels=list(dirc.glob('*'))
    for i in range(len(labels)):
#         putting the name of folder inside main dir in key of dict 
#         putting all the images in the folder in list , in value of dict
        images_dict[str(labels[i]).split('/')[-2]]=list(labels[i].parent.glob('*'))
    return images_dict

get_images1(list(data_dir.glob('*/*'))[:20])

In [ ]:
path_list=list(data_dir.glob('*/*'))[:200]
plant_2kimages_dict=get_images1(path_list)

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:
# processing the images into a list for training set 
# which will be used for training the model

def img_read(image_dict,label_dict):
    x=[]
    y=[]
    sum1=0
    for nameid,images in image_dict.items():
        for image in images:
            img=cv2.imread(str(image))
            resized_img=cv2.resize(img,(512,512))
            x.append(resized_img)
            y.append(label_dict[nameid])
            sum1=sum1+1
            print('number : ',sum1)
        
    x=np.array(x)
    y=np.array(y)
    return x,y

In [24]:
count=0
for i,j in plant_2kimages_dict.items():
    for k in j:
        count+=1
    
count

296

In [ ]:
X,y=img_read(plant_2kimages_dict,plant_all_label_dict)

In [ ]:
# from sklearn.model_selection import train_test_split

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# `pretrained model with input image shape 512x512 , without final layer`

In [3]:
pre_trained_model=hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2",
                                 input_shape=(512, 512, 3), trainable=True)


In [7]:
num_of_plants = len(plant_all_labels_dict)

NameError: name 'plant_all_labels_dict' is not defined

In [5]:
model=keras.models.Sequential(
    pre_trained_model,
    layers.Dense(80000)
)

In [8]:
model.compile()

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [12]:
from joblib import load,dump

dump(model,filename='./models/model_pre_efficientnet_v2_imagenet21k_ft1k_xl.joblib')

['./models/model_pre_efficientnet_v2_imagenet21k_ft1k_xl.joblib']

In [ ]:
loaded_model=load(filename='model_pre_name.joblib')